<a href="https://colab.research.google.com/github/Justus-coded/Competitions/blob/master/DSN%20AI%20Bootcamp%20Qualification%20Hackathon/DSN%20AI%20Boot_camp%20Qualification1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
 
%matplotlib inline
 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#train = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Train.csv')
#test  = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Test.csv')

train = pd.read_csv('../input/dsn-ai-bootcamp-data/Train.csv')
test = pd.read_csv('../input/dsn-ai-bootcamp-data/Test.csv')
sub = pd.read_csv('../input/dsn-ai-bootcamp-data/SampleSubmission.csv')

In [ ]:
dict_ = {'charge': 1,
         'lending': 0}
 
target_map = {
    'yes': 1,
    'no': 0}
 
train.default_status.replace(target_map, inplace = True)
 
train['form_field47'].replace(dict_, inplace = True)
test['form_field47'].replace(dict_, inplace = True)

In [ ]:
#pip install catboost

In [ ]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

In [ ]:
features = train.select_dtypes(exclude = object).columns.drop(['default_status'])

In [ ]:
# features

In [ ]:
for col in features:
    train[col].fillna(-999, inplace = True)
    test[col].fillna(-999, inplace = True)

In [ ]:
train['form_field1*formfield2'] = train['form_field1'] * train['form_field2']
train['form_field1/formfield2'] = train['form_field1'] / train['form_field2']

train['form_field1*formfield3'] = train['form_field1'] * train.form_field3
train['form_field1*formfield4'] = train.form_field1 * train.form_field4
train['form_field1*formfield5'] = train.form_field1 * train.form_field5

#new_train['form_field30*formfield41'] = new_train['form_field30'] * new_train['form_field41']
##new_train['form_field31*formfield40'] = new_train['form_field31'] * new_train['form_field40']

train['form_field6+field7'] =  (train.form_field6 + train.form_field8)/2

train['form_field1*formfield6'] = train['form_field1'] * train['form_field6']
train['form_field2*formfield6'] = train['form_field2'] * train['form_field6']

train['form_field1*formfield9'] = train['form_field1'] * train['form_field9']
train['form_field2*formfield9'] = train['form_field2'] * train['form_field9']

train['form_field1*formfield37'] = train['form_field1'] * train['form_field37']
train['form_field2*formfield37'] = train['form_field2'] * train['form_field37']

train['form_field9*formfield6'] = train['form_field9'] * train['form_field6']
train['form_field37*formfield6'] = train['form_field37'] * train['form_field6']

train['form_field9*formfield37'] = train['form_field9'] * train['form_field37']
train['form_field1*formfield8'] = train['form_field1'] * train['form_field8']

In [ ]:
test['form_field1*formfield2'] = test['form_field1'] * test['form_field2']
test['form_field1/formfield2'] = test['form_field1'] / test['form_field2']

test['form_field1*formfield3'] = test['form_field1'] * test.form_field3
test['form_field1*formfield4'] = test.form_field1 * test.form_field4
test['form_field1*formfield5'] = test.form_field1 * test.form_field5

test['form_field6+field7'] =  (test.form_field6 + test.form_field8)/2

test['form_field1*formfield6'] = test['form_field1'] * test['form_field6']
test['form_field2*formfield6'] = test['form_field2'] * test['form_field6']

test['form_field1*formfield9'] = test['form_field1'] * test['form_field9']
test['form_field2*formfield9'] = test['form_field2'] * test['form_field9']

test['form_field1*formfield37'] = test['form_field1'] * test['form_field37']
test['form_field2*formfield37'] = test['form_field2'] * test['form_field37']

test['form_field9*formfield6'] = test['form_field9'] * test['form_field6']
test['form_field37*formfield6'] = test['form_field37'] *test['form_field6']

test['form_field9*formfield37'] = test['form_field9'] * test['form_field37']
test['form_field1*formfield8'] = test['form_field1'] * test['form_field8']

In [ ]:
X = train[features]
y = train['default_status']

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
def metric(y, pred):
    return roc_auc_score(y, pred, labels=[0, 1])

In [ ]:
# Specify number of folds
n_skf = 10
kf = StratifiedKFold(n_skf)
 
 
seed = 42
 
## Manually tweak the hyperparameter, learning rate and n_estimators especially
params = {'max_depth':7,
    'n_estimators': 5000,
    'learning_rate': 0.008,
    'objective': 'CrossEntropy',
      'eval_metric':'AUC',
    'random_seed': seed,
    'early_stopping_rounds': 200,
    'use_best_model': True,
}

In [ ]:
score_list = []
score = 0
test_oofs = []
 
for i, (tr_idx, vr_idx) in enumerate(kf.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]
 
    model = CatBoostClassifier(**params, task_type='GPU')
    model.fit(xtrain, ytrain, eval_set=[(xval,yval)], verbose=100)
    
    p = model.predict_proba(xval)[:, 1]
    sc = metric(yval, p)
    score_list.append(sc)
    score += sc/n_skf
    
    pred = model.predict_proba(test[features])[:, 1]
    test_oofs.append(pred)
 
    print('Fold {} : {}'.format(i, sc))
 
print()
print()
print('Avg log : ', score)

0:	learn: 0.8005818	test: 0.7965800	best: 0.7965800 (0)	total: 18ms	remaining: 1m 29s
100:	learn: 0.8311888	test: 0.8169689	best: 0.8169994 (99)	total: 1.58s	remaining: 1m 16s
200:	learn: 0.8370891	test: 0.8215050	best: 0.8215050 (200)	total: 3.13s	remaining: 1m 14s
300:	learn: 0.8409205	test: 0.8239534	best: 0.8239534 (300)	total: 4.7s	remaining: 1m 13s
400:	learn: 0.8439492	test: 0.8253606	best: 0.8253606 (400)	total: 6.2s	remaining: 1m 11s
500:	learn: 0.8465233	test: 0.8262982	best: 0.8262982 (500)	total: 7.7s	remaining: 1m 9s
600:	learn: 0.8490306	test: 0.8273727	best: 0.8273727 (600)	total: 9.23s	remaining: 1m 7s
700:	learn: 0.8511631	test: 0.8281304	best: 0.8281304 (700)	total: 11.2s	remaining: 1m 8s
800:	learn: 0.8531602	test: 0.8286533	best: 0.8286559 (799)	total: 13.2s	remaining: 1m 9s
900:	learn: 0.8550197	test: 0.8290893	best: 0.8290893 (900)	total: 14.8s	remaining: 1m 7s
1000:	learn: 0.8567368	test: 0.8295559	best: 0.8295683 (999)	total: 16.3s	remaining: 1m 5s
1100:	learn: 

1800:	learn: 0.8697619	test: 0.8405893	best: 0.8405893 (1800)	total: 30.2s	remaining: 53.6s
1900:	learn: 0.8714205	test: 0.8407020	best: 0.8407056 (1899)	total: 31.8s	remaining: 51.9s
2000:	learn: 0.8729822	test: 0.8407680	best: 0.8407773 (1996)	total: 34.1s	remaining: 51s
2100:	learn: 0.8745603	test: 0.8407979	best: 0.8408132 (2090)	total: 35.8s	remaining: 49.4s
2200:	learn: 0.8761445	test: 0.8408687	best: 0.8408927 (2179)	total: 37.4s	remaining: 47.6s
2300:	learn: 0.8777120	test: 0.8408967	best: 0.8408967 (2300)	total: 39.2s	remaining: 45.9s
2400:	learn: 0.8792587	test: 0.8408776	best: 0.8409409 (2347)	total: 40.8s	remaining: 44.2s
2500:	learn: 0.8807765	test: 0.8409327	best: 0.8409527 (2499)	total: 42.4s	remaining: 42.3s
2600:	learn: 0.8822640	test: 0.8409133	best: 0.8409527 (2499)	total: 44s	remaining: 40.6s
2700:	learn: 0.8836103	test: 0.8409688	best: 0.8409710 (2632)	total: 46s	remaining: 39.2s
2800:	learn: 0.8851086	test: 0.8409791	best: 0.8409924 (2791)	total: 47.6s	remaining: 

3600:	learn: 0.8949020	test: 0.8452739	best: 0.8453799 (3557)	total: 1m 1s	remaining: 23.9s
3700:	learn: 0.8962035	test: 0.8451736	best: 0.8453799 (3557)	total: 1m 3s	remaining: 22.1s
bestTest = 0.845379889
bestIteration = 3557
Shrink model to first 3558 iterations.
Fold 4 : 0.8453799000648331
0:	learn: 0.8012357	test: 0.7934641	best: 0.7934641 (0)	total: 16.5ms	remaining: 1m 22s
100:	learn: 0.8302389	test: 0.8267238	best: 0.8267238 (100)	total: 1.56s	remaining: 1m 15s
200:	learn: 0.8362135	test: 0.8310861	best: 0.8310861 (200)	total: 3.12s	remaining: 1m 14s
300:	learn: 0.8402775	test: 0.8339104	best: 0.8339104 (300)	total: 5.18s	remaining: 1m 20s
400:	learn: 0.8432067	test: 0.8358301	best: 0.8358301 (400)	total: 6.76s	remaining: 1m 17s
500:	learn: 0.8458682	test: 0.8370883	best: 0.8370883 (500)	total: 8.33s	remaining: 1m 14s
600:	learn: 0.8482124	test: 0.8379809	best: 0.8379809 (600)	total: 9.94s	remaining: 1m 12s
700:	learn: 0.8503265	test: 0.8387557	best: 0.8387557 (700)	total: 11.5

2600:	learn: 0.8814799	test: 0.8435932	best: 0.8435996 (2569)	total: 43.8s	remaining: 40.4s
2700:	learn: 0.8828673	test: 0.8435563	best: 0.8436093 (2628)	total: 45.9s	remaining: 39.1s
2800:	learn: 0.8842677	test: 0.8435819	best: 0.8436093 (2628)	total: 48.3s	remaining: 37.9s
bestTest = 0.8436092734
bestIteration = 2628
Shrink model to first 2629 iterations.
Fold 7 : 0.8436092439408525
0:	learn: 0.7974082	test: 0.8190478	best: 0.8190478 (0)	total: 16.9ms	remaining: 1m 24s
100:	learn: 0.8292123	test: 0.8462776	best: 0.8462776 (100)	total: 1.65s	remaining: 1m 19s
200:	learn: 0.8349121	test: 0.8495482	best: 0.8495482 (200)	total: 3.28s	remaining: 1m 18s
300:	learn: 0.8387992	test: 0.8517312	best: 0.8517334 (298)	total: 4.93s	remaining: 1m 17s
400:	learn: 0.8418957	test: 0.8529946	best: 0.8529946 (400)	total: 6.61s	remaining: 1m 15s
500:	learn: 0.8445688	test: 0.8538873	best: 0.8538873 (500)	total: 8.62s	remaining: 1m 17s
600:	learn: 0.8469189	test: 0.8545760	best: 0.8545882 (598)	total: 10

In [ ]:
f"{n_skf} fold CV, score: {score}" #Increase in local cv score should probably lead to increase in Leaderboard score

'10 fold CV, score: 0.8416283637107365'

'10 fold CV, score: 0.8416283637107365', LR-0.008, n_estimator = 5000, seed = 42, depth = 7, submit5.csv

'10 fold CV, score: 0.8404761359972677', LR-0.008, n_estimator = 5000, seed = 42, depth = 10, submit4.csv

'10 fold CV, score: 0.8416127641119365', LR-0.008, n_estimator = 5000, seed = 2020

'10 fold CV, score: 0.8415039416559706', LR-0.008, n_estimator = 5000, seed = 42

### Since we are using cross validation, We'll have `n` different predictions (where n is the number of fold)

In [ ]:
oof_prediction = pd.DataFrame(test_oofs).T

In [ ]:
oof_prediction.columns = ['fold_'+ str(i) for i in range(1, n_skf + 1)]

In [ ]:
oof_prediction.head()

,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7,fold_8,fold_9,fold_10
0,0.303809,0.325865,0.317122,0.295998,0.326069,0.335429,0.313768,0.315341,0.293900,0.319507
1,0.355574,0.381696,0.321014,0.378279,0.401322,0.367036,0.387918,0.350336,0.398904,0.364839
2,0.368070,0.363516,0.399437,0.415677,0.396283,0.401986,0.395054,0.384818,0.350068,0.370680
3,0.734844,0.753176,0.776929,0.770310,0.753804,0.757743,0.763865,0.745151,0.764903,0.753553
4,0.156001,0.183129,0.181449,0.157846,0.171436,0.176964,0.171920,0.161259,0.184253,0.168552


In [ ]:
#sub = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/SampleSubmission.csv')

In [ ]:
### We take the mean of all 5 predictions to generate submission

In [ ]:
sub['default_status'] = np.mean(test_oofs, axis = 0)

In [ ]:
sub.to_csv('submit5.csv', index = False)

### Suggestions:
- Base your feature selection on the local cv score i.e When you add, remove or engineer feature check `score` (cell 17)
- Increase number of fold to 10, 15. See how score imporve on leaderboard.
- Try differnt value of learning rate. I'll advice anything < 0.01
- Create as many features as possible.
- Find code for Xgboost and lgbm using this method of modelling, or better still tweak this one (mind you, names of parameters may be slightly different.